## AgentCore Evaluations - LangGraph를 위한 온디맨드 평가

이 튜토리얼에서는 LangGraph agent에 적용되는 AgentCore Evaluations의 온디맨드 평가를 사용하는 방법을 배웁니다.

이 실습을 실행하려면 먼저 [00-prereqs](../../00-prereqs) 폴더의 코드를 사용하여 LangGraph agent를 생성하고 [01-creating-custom-evaluators](../../01-creating-custom-evaluators)의 코드를 사용하여 커스텀 evaluator를 생성해야 합니다.

### 학습 내용
- AgentCore Starter toolkit을 사용하여 trace에 대한 온디맨드 평가를 실행하는 방법

### 튜토리얼 세부 정보

| Information         | Details                                                                       |
|:--------------------|:------------------------------------------------------------------------------|
| Tutorial type       | 온디맨드 evaluator(내장 및 커스텀)를 사용한 LangGraph agent 평가      |
| Tutorial components | 내장 및 커스텀 evaluator를 사용한 평가 실행                        |
| Tutorial vertical   | Cross-vertical                                                                |
| Example complexity  | Easy                                                                          |
| SDK used            | Amazon Bedrock AgentCore Starter toolkit                                      |

### 온디맨드 평가

온디맨드 평가는 선택한 span 세트를 직접 분석하여 특정 agent 상호작용을 평가하는 유연한 방법을 제공합니다. 프로덕션 트래픽을 지속적으로 모니터링하는 온라인 평가와 달리, 온디맨드 평가를 사용하면 언제든지 선택한 상호작용에 대한 타겟 평가를 수행할 수 있습니다.

온디맨드 평가를 사용하면 span, trace 또는 session ID를 제공하여 평가하려는 정확한 span, trace 또는 session을 지정합니다. AgentCore Starter toolkit을 사용하면 session의 모든 trace를 자동으로 평가할 수도 있습니다.

그런 다음 커스텀 evaluator 또는 내장 evaluator를 agent의 상호작용에 적용할 수 있습니다. 이 평가 유형은 특정 고객 상호작용을 조사하거나, 보고된 문제에 대한 수정 사항을 검증하거나, 품질 개선을 위해 과거 데이터를 분석해야 할 때 특히 유용합니다. 평가 요청을 제출하면 서비스는 지정된 span만 처리하고 분석을 위한 상세한 결과를 제공합니다.

### Agent에서 AgentCore Observability로 trace 생성

AgentCore Observability는 상세한 실행 데이터를 캡처하고 구조화하기 위한 기반으로 [OpenTelemetry (OTEL)](https://opentelemetry.io/) trace를 활용하여 호출 중 agent 동작에 대한 포괄적인 가시성을 제공합니다. AgentCore는 [AWS Distro for OpenTelemetry (ADOT)](https://aws-otel.github.io/)를 사용하여 다양한 agent 프레임워크에서 여러 유형의 OTEL trace를 계측합니다.

Agent가 AgentCore Runtime에서 호스팅되는 경우(이 튜토리얼의 agent처럼), AgentCore Observability 계측은 최소한의 구성으로 자동으로 수행됩니다. `requirements.txt`에 `aws-opentelemetry-distro`를 포함하기만 하면 AgentCore Runtime이 OTEL 구성을 자동으로 처리합니다. Agent가 AgentCore Runtime에서 실행되지 않는 경우, AgentCore Observability에서 사용할 수 있도록 ADOT로 계측해야 합니다. 원격 측정 데이터를 CloudWatch로 전달하도록 환경 변수를 구성하고 OpenTelemetry 계측으로 agent를 실행해야 합니다.

프로세스는 다음과 같습니다:

![session_traces](../../images/observability_traces.png)

Session trace가 AgentCore Observability에서 사용 가능해지면 AgentCore Evaluations를 사용하여 agent의 동작을 평가할 수 있습니다.

### 온디맨드 평가가 trace와 작동하는 방식

온디맨드 평가에서 agent가 호출되고 AgentCore Observability에서 trace를 생성합니다. 이러한 trace는 session에 매핑되고 로그는 Amazon CloudWatch Log 그룹에서 사용할 수 있게 됩니다. 온디맨드 평가를 사용하면 개발자가 사용할 session 또는 trace를 결정하고 trace 콘텐츠를 평가할 메트릭과 함께 이를 AgentCore Evaluations에 입력으로 보냅니다. 프로세스는 다음과 같습니다:


![session_traces](../../images/on_demand_evaluations.png)

### 이전 튜토리얼에서 정보 검색

이 튜토리얼에서는 사전 요구 사항 튜토리얼에서 AgentCore Runtime에 배포된 LangGraph agent를 사용합니다. 사전 구축된 메트릭과 `01-creating-custom-metrics` 튜토리얼에서 생성한 `response_quality` 메트릭으로 평가합니다. agent 및 evaluator 정보를 검색해 보겠습니다.

In [ ]:
# 이전 노트북에서 저장된 변수들을 불러옴
%store -r launch_result_langgraph
%store -r session_id_langgraph
%store -r evaluator_id
try:
    print("Agent Id:", launch_result_langgraph.agent_id)
    print("Agent ARN:", launch_result_langgraph.agent_arn)
except NameError as e:
    raise Exception("""Missing launch results from your LangGraph agent. Please run 00-prereqs before executing this lab""")

try:
    print("Session id:", session_id_langgraph)
except NameError as e:
    raise Exception("""Missing session id from your LangGraph agent. Please run 00-prereqs before executing this lab""")

try:
    print("Evaluator id:", evaluator_id)
except NameError as e:
    raise Exception("""Missing custom evaluator id. Please run 01-creating-custom-evaluators before executing this lab""")

### AgentCore Evaluations 클라이언트 초기화

이제 AgentCore Starter toolkit에서 AgentCore Evaluations 클라이언트를 초기화해 보겠습니다.

In [ ]:
from bedrock_agentcore_starter_toolkit import Evaluation, Observability
import os
import json
from boto3.session import Session
from IPython.display import Markdown, display

In [ ]:
# AWS 세션 생성 및 리전 정보 가져오기
boto_session = Session()
region = boto_session.region_name
print(region)

In [ ]:
# AgentCore Evaluation 클라이언트 초기화
eval_client = Evaluation(region=region)

### 평가 실행

AgentCore Evaluations를 실행하려면 session, trace 또는 span 정보를 제공해야 합니다. 이전 튜토리얼에서 본 것처럼 다양한 메트릭은 agent trace에서 다양한 수준의 정보를 요구합니다.

![metrics level](../../images/metrics_per_level.png)

AWS의 SDK 중 하나를 사용하는 경우 trace를 직접 처리해야 합니다. AgentCore Starter toolkit은 이 프로세스를 단순화하고 session id 또는 trace id를 기반으로 trace를 처리합니다. 이렇게 하면 `run` 상호작용 중에 AgentCore starter toolkit evaluator 클라이언트에 session id를 제공할 수 있으며 SDK는 trace 및 span 수준 메트릭에 대해 해당 session의 모든 trace를 추출하고 평가합니다. 선택적으로 평가 결과를 저장할 출력 파일의 이름을 제공할 수도 있습니다.

### Goal Success Rate

이제 agent의 Goal Success Rate를 평가해 보겠습니다. agent에게 다음 질문을 했던 것을 기억하세요:

* What is the weather now?
* How much is 2+2?
* Can you tell me the capital of the US?

In [ ]:
# Goal Success Rate 평가 실행 (session 레벨 메트릭)
goal_sucess_results = eval_client.run(
    agent_id=launch_result_langgraph.agent_id,
    session_id=session_id_langgraph, 
    evaluators=["Builtin.GoalSuccessRate"]
)

이제 evaluator의 결과를 이해해 보겠습니다. 결과 객체에는 수행된 평가에 대한 정보(session_id, trace_id, input_data)와 평가 결과가 포함됩니다.

평가 결과에는 evaluator 정보(id, name, ARN), 평가 값, 평가 레이블, 평가 설명 및 평가 작업에 대한 일부 추가 컨텍스트(spanContext, token_usage, ...)가 포함됩니다.

평가 응답을 살펴보겠습니다

In [ ]:
# 평가 결과 출력 (값, 설명, 토큰 사용량, 컨텍스트)
for result in goal_sucess_results.results:
    information = f"""
    Goal Success: {result.label} ({result.value})
    Explanation: \n{result.explanation}]\n
    Token Usage: {result.token_usage}\n
    Context: {result.context}\n
    """
    display(Markdown(information))

### Correctness

이제 trace 수준 메트릭인 Correctness에 대해 동일한 session을 분석해 보겠습니다

In [ ]:
# Correctness 평가 실행 (trace 레벨 메트릭)
correctness_results = eval_client.run(
    agent_id=launch_result_langgraph.agent_id,
    session_id=session_id_langgraph, 
    evaluators=["Builtin.Correctness"]
)

이제 evaluator의 결과를 이해해 보겠습니다. 이 경우 correctness는 trace 수준에서 평가되므로 각 trace는 자체 평가 결과를 얻습니다.

In [ ]:
# 각 trace별 correctness 결과 출력
for result in correctness_results.results:
    information = f"""
    Correctness: {result.label} ({result.value})
    Explanation: \n{result.explanation}]\n
    Token Usage: {result.token_usage}\n
    Context: {result.context}\n
    """
    display(Markdown(information))
    print("================================================")

### Tool selection accuracy 및 parameter selection accuracy

이제 tool 및 parameter 선택에 대해 agent를 평가해 보겠습니다. 두 메트릭 모두 span 수준에서 평가됩니다.

In [ ]:
# Tool 선택 및 Parameter 선택 정확도 평가 (span 레벨 메트릭)
parameter_results = eval_client.run(
    agent_id=launch_result_langgraph.agent_id,
    session_id=session_id_langgraph, 
    evaluators=["Builtin.ToolParameterAccuracy", "Builtin.ToolSelectionAccuracy"]
)

이제 결과를 분석해 보겠습니다. 이 경우 동일한 실행에서 두 가지 다른 메트릭으로 session을 평가하고 있습니다. 즉, 이제 어떤 evaluator가 각 응답을 생성하는지 알아야 합니다. 결과의 `evaluator_name` 속성으로 이를 수행할 수 있습니다. agent가 tool을 얼마나 잘 사용했는지 살펴보겠습니다:

In [ ]:
# 여러 evaluator 결과 출력 (evaluator_name으로 구분)
for result in parameter_results.results:
    information = f"""
    Metric: {result.evaluator_name}
    Value: {result.label} ({result.value})
    Explanation: \n{result.explanation}]\n
    Token Usage: {result.token_usage}\n
    Context: {result.context}\n
    """
    display(Markdown(information))
    print("================================================")

### 커스텀 evaluator 사용

이제 session, trace 및 span 수준에서 evaluator를 사용했으므로 커스텀 메트릭을 사용하여 응답 품질을 평가해 보겠습니다:

In [ ]:
# 커스텀 evaluator로 응답 품질 평가
custom_results = eval_client.run(
    agent_id=launch_result_langgraph.agent_id,
    session_id=session_id_langgraph, 
    evaluators=[evaluator_id]
)

이제 평가 결과를 살펴보겠습니다. 이 경우 다음 지침이 있는 agent를 평가하고 있습니다:

```
You're a helpful assistant. You can do simple math calculation, and tell the weather.
```

평가 메트릭의 경우 평가 지침에 명시된 대로 범위를 벗어난 agent에 대해 `Very Poor` 품질로 페널티를 부과합니다:

```
...
**IMPORTANT**: A response quality can only be high if the agent remains in its original scope. Penalize agents that answer questions outside its original scope with a Very Poor classification.
...
```

다음 질문을 평가하고 있으므로:

* What is the weather now?
* How much is 2+2?
* Can you tell me the capital of the US?

마지막 질문에 대해 agent가 `Very Poor` 평가를 받을 것으로 예상합니다.

In [ ]:
# 커스텀 evaluator 결과 출력
for result in custom_results.results:
    information = f"""
    Metric: {result.evaluator_name}
    Value: {result.label} ({result.value})
    Explanation: \n{result.explanation}]\n
    Token Usage: {result.token_usage}\n
    Context: {result.context}\n
    """
    display(Markdown(information))
    print("================================================")

### 평가 결과 저장

AgentCore starter toolkit은 구조화된 출력 파일에 agent 평가 결과를 저장하는 데도 도움이 됩니다. 이를 위해 실행 중에 `ouput` 매개변수를 제공하기만 하면 됩니다.

In [ ]:
# 평가 결과를 JSON 파일로 저장 (output 파라미터 사용)
save_results = eval_client.run(
    agent_id=launch_result_langgraph.agent_id,
    session_id=session_id_langgraph, 
    evaluators=[
        evaluator_id
    ],
    output="evals_results/output.json"
)

### 축하합니다!

이제 온디맨드 기능으로 agent를 평가했습니다. 다음 튜토리얼에서는 온라인 evaluator를 설정하고 agent와 연결하여 프로덕션 환경에서 agent 평가를 자동화합니다.